In [35]:
import pandas as pd
import numpy as np

class SavingsA:
    """
    Name: Savings Report
    Author: GuoRubing
    Date: Nov. 17, 2017
    input: Summary, IM, Price1, Cross1
    output: Savings
    """
    def __init__(self, sav_dataframe, im_dataframe,):
        self.sav_df = sav_dataframe
        self.im_df = im_dataframe
        
    def sav_1(self):
        """
        :type my_dataframe: DataFrame
        :rtype: DataFrame
        """
        foo = self.sav_df[:]
        foo = foo[np.isfinite(foo["Item No"])]
        foo["Item No"] = foo["Item No"].astype(int).astype(str)
        foo['SAP+UOM'] = foo["Item No"].str.cat(foo["U O M"], sep = '-')
        #foo["EA Price"] = foo["Last Price"]/foo["EA Conversion"]
        return foo
    def imMergeSav_1(self):
        df_im = self.im_df.loc[:]
        df_im["OrderUoM"] = df_im["OUoM"].combine_first(df_im["BUoM"]) 
        df_im["OUoM"].fillna(df_im["BUoM"], inplace=True)
        df_im["Material Number"]=df_im["Material Number"].fillna(0).astype(float).astype(int).astype(str)
        df_im["SAP+UOM"] = df_im["Material Number"].str.cat(df_im["OUoM"], sep = '-')
        df_im["Conversion Factor"] = df_im["ConvOrder"]*df_im["PyxConvLow"]
        im_s1 = df_im['SAP+UOM']
        im_s2 = df_im["Conversion Factor"]
        df_im_Rng = pd.concat([im_s1, im_s2], axis=1)
        Merge1 = pd.merge(self.sav_1(), df_im_Rng, left_on = "SAP+UOM", right_on = "SAP+UOM", how = "left")
        Merge1["EA Conversion"] = Merge1["Conversion Factor"]
        Merge1["EA Price"] = Merge1["Last Price"]/Merge1["EA Conversion"]
        Merge1["EA Qty"] = Merge1["Sum of Qty Paid"]/Merge1["EA Conversion"]
        return Merge1
    def price(self):
        self.imMergeSav_1()

In [88]:
xlsx_file = "SAP_Sterilization_Solutions.xlsx"
#df1=pd.read_csv(csv_file)
#Show pd.read_csv details
#help(pd.read_csv)
#help(pd.read_excel)
df_sav = pd.read_excel(xlsx_file, sheetname="Savings")
df_im = pd.read_excel(xlsx_file, sheetname="IM")
df_price = pd.read_excel(xlsx_file, sheetname = "Price1")
a = SavingsA(df_sav, df_im)
print(a.imMergeSav_1())

  Manufacturer Name Manufacturer Part No  \
0         J & J ASP              AS20390   
1            METREX              10-6000   
2            METREX              10-6000   
3            METREX               10-602   

                                  Last Desc  \
0     SOLUTION DISINFECT CIDEX OPA 1 GALLON   
1  DISINFECTANT OPA + MTRCD 1GL INST RTU HI   
2          SOLUTION METRICIDE OPA PLUS 4/CS   
3          STRIP DSINF TST MTRCD OPA + MNTR   

                          Vendor Name Vendor Mat Number U O M Item No  \
0  MEDLINE INDUSTRIES (OFF-SITE ONLY)         J-J20390H   GLL   24060   
1              MEDLINE INDUSTRIES INC         MAP106000    CS   10234   
2           HENRY SCHEIN INCORPORATED           6408231    CS   24064   
3              MEDLINE INDUSTRIES INC          MAP10602    CS   19066   

   Last Price  Sum of Qty Paid  Sum of Total    SAP+UOM  Conversion Factor  \
0       66.71               16       1067.36  24060-GLL                  1   
1       80.00        

In [121]:
# Price Macro

"""
def sav_1(df):
    foo = df[:]
    
    foo = foo[np.isfinite(df['Item No'])]
    #df_sav["Item No"] = df_sav["Item No"].fillna(0).astype(float).astype(int).astype(str)
    foo["Item No"] = foo["Item No"].astype(float).astype(int).astype(str)
    foo["SAP+UOM"] = foo["Item No"].str.cat(foo["U O M"], sep = '-')
    return foo
sav_1(df_sav)
"""
xl = pd.ExcelFile(xlsx_file)
#xl.parse(sheet_name)
#x=['Price1', 'Price2','Price3', 'Price1']
def PriceCross(xl):
    
    #xl.sheet_names  # see all sheet names
    v_dict = {}
    cp_pair = []
    priceRng_df = pd.DataFrame()
    crossRng_df = pd.DataFrame()
    #for sheet_name in xl.sheet_names:
    for sheet_name in xl.sheet_names:
        #print(sheet_name,)
        if '1' in sheet_name:
            if "Price" in sheet_name: 
                #v_dict[sheet_name] = pd.read_excel(xlsx_file, sheetname=sheet_name)
                df1 = pd.read_excel(xlsx_file, sheetname=sheet_name)
                vendor_name = df1["SupplierName"][0]
                #d[vendor_name] = d[sheet_name]
                #print (d[sheet_name]["ProductName"])
                priceRng_df = df1.loc[:, 'ProductName':'Price']
                priceRng_df["EA Price"] = priceRng_df["Price"]/priceRng_df["EqualsLesserUOMQuantity"]
                cp_pair.append(priceRng_df)
                v_dict[vendor_name] = cp_pair
                price_df_list.append(sheet_name)
                return v_dict
            elif "Cross" in sheet_name:
                v_dict[sheet_name] = pd.read_excel(xlsx_file, sheetname=sheet_name)
                crossRng_df = v_dict[sheet_name].loc[:, 'ProductName':'Price']
print(PriceCross(xl))

{'Metrex Research Corp': [                                         ProductName MfgProductId  \
0  METRICIDE 2.6% ACTIVATED DIALDEHYDE SOLUTION  ...      10-1400   
1  METRICIDE 2.6% ACTIVATED DIALDEHYDE 14 DAY (QU...      10-1405   
2                DISINFECTANT, METRICIDE LNG LIFE GL      10-2800   
3  METRICIDE 28 2.5% LONG-LIFE ACTIVATED DIALDEHY...      10-2805   
4                              METRI-TEST 1.5% STRIP       10-303   
5                         METRISTRIP 10-304   (2/CS)       10-304   
6                                          METRIZYME      10-4000   
7                                        METRISPONGE      10-4025   
8                                 METRICIDE OPA PLUS      10-6000   
9                            MET OPA PLUS TEST STRIP       10-602   

   DistributorProductID UOMCode GreaterUOMCode  EqualsLesserUOMQuantity  \
0                   NaN      CA             CA                        4   
1                   NaN      CA             CA                  

In [96]:
d = {}
def PriceMacro(sheet):
    vendor_name = sheet["SupplierName"][0]
    sheet[vendor_name] = 
    d[sheet_name] = pd.read_excel(xlsx_file, sheetname=sheet_name)
    print(vendor_name)
PriceMacro(price_df_list[0])
#print(vendor_name)

SyntaxError: invalid syntax (<ipython-input-96-857fbd077a7d>, line 4)

In [112]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
#print (x['x'][0])
x['v1']=x['x']
print(x)
print (x.loc[:,'y':'v1'])

   x  y  v1
0  1  3   1
1  2  4   2
2  3  5   3
   y  v1
0  3   1
1  4   2
2  5   3


In [8]:
df = pd.DataFrame(np.random.randn(4, 2)*3, columns=['a', 'b'])
df.style.format("{:.2}")
df['c'] = ['a', 'b', 'c', 'd']
df.style.format({'C': str.upper})

,a,b,c
0,0.292139,-7.34445,a
1,3.89978,-3.11239,b
2,1.23942,-1.12177,c
3,3.82345,-5.41029,d


In [137]:
def im(df):
    df_im = df.loc[:]
    df_im["OrderUoM"] = df_im["OUoM"].combine_first(df_im["BUoM"]) 
    df_im["OUoM"].fillna(df_im["BUoM"], inplace=True)
    df_im["Material Number"]=df_im["Material Number"].fillna(0).astype(float).astype(int).astype(str)
    df_im["SAP+UOM"] = df_im["Material Number"].str.cat(df_im["OUoM"], sep = '-')
    #type(df_im["Material Number"])
    df_im["Conversion Factor"] = df_im["ConvOrder"]*df_im["PyxConvLow"]
    im_s1 = df_im['SAP+UOM']
    im_s2 = df_im["Conversion Factor"]
    df_im_Rng = pd.concat([im_s1, im_s2], axis=1)
    return df_im_Rng
im(df_im)

,SAP+UOM,Conversion Factor
0,10234-CS,4
1,19066-CS,2
2,24060-GLL,1
3,24064-CS,1
4,27359-GLL,1
5,47606-EA,1


In [ ]:
# Sav_1
#df1['Item No'].fillna(0)
df_sav["Item No"] = df_sav["Item No"].fillna(0).astype(float).astype(int).astype(str)
#df_sav["Item No"]

#ser = pd.Series([1,2], dtype = 'int64')
#ser.astype('str')
#df_sav["Manufacturer Part No"]
df_sav["SAP+UOM"] = df_sav["Item No"].str.cat(df_sav["U O M"], sep = '-')


# IM_Macro VBA:
df_im.head()
#df_im.dtypes
df_im["OrderUoM"] = df_im["OUoM"].combine_first(df_im["BUoM"]) 

df_im["OUoM"].fillna(df_im["BUoM"], inplace=True)
df_im["Material Number"]=df_im["Material Number"].fillna(0).astype(float).astype(int).astype(str)
df_im["SAP+UOM"] = df_im["Material Number"].str.cat(df_im["OUoM"], sep = '-')
type(df_im["Material Number"])
df_im["Conversion Factor"] = df_im["ConvOrder"]*df_im["PyxConvLow"]
df_im

im_s1 = df_im['SAP+UOM']
im_s2 = df_im["Conversion Factor"] 
df_im_Rng = pd.concat([im_s1, im_s2], axis=1)
df_sav = pd.merge(df_sav, df_im_Rng, left_on = 'SAP+UOM', right_on = 'SAP+UOM', how='left')

df_sav.dtypes

In [89]:
dfc = pd.DataFrame({'A':['aaa','bbb','ccc'],'B':[1,2,3]})

In [90]:
dfc

,A,B
0,aaa,1
1,bbb,2
2,ccc,3


In [91]:
dfc.loc[0,'A'] = 11

In [94]:
dfc.loc[:]

,A,B
0,11,1
1,bbb,2
2,ccc,3
